In [1]:
from __future__ import print_function
from ortools.constraint_solver import pywrapcp

In [2]:
people = [
        'Alexey Alekseev',
        'Alexey Mironov',
        'Alexey Eskov',
        'Alexey Evseev',
        'Anton Rodin',
        'Arshdeep Singh',
        'Egor Sachko',
        'Erik Hinterbichler',
        'Danyi Gergely',
        'Ihor Bykov',
        'Ilya Tikhonov',
        'Kirill Klokov',
        'Luis del Giudice Sierra',
        'Mikael Arakelian',
        'Nikolay Fokin',
        'Oleg Khomyuk',
        'Oleksii Solianik',
        'Owais Lone',
        'Spyridon Oikonomakis',
        'Tigran Najaryan',
        'Vadim Goncharov'
]

In [3]:
snoring_people = {
        'Oleg Khomyuk', 'Owais Lone', 'Luis del Giudice Sierra', 'Ilya Tikhonov',
    }

In [6]:
hates_snores = {
        'Tigran Najaryan', 'Erik Hinterbichler', 'Alexey Eskov'
}

In [7]:
constraints = [
        (hates_snoring, snores)
        for hates_snoring in hates_snores
        for snores in snoring_people
]
n = len(people)

In [8]:
person_index = {
    person: index
    for index, person in enumerate(people)
}
index_to_person = dict(
    list(enumerate(people))
)

In [9]:
solver = pywrapcp.Solver('room splitting optimization')

In [10]:
neighbour_vars = [
    solver.IntVar(
        0, n - 1, 'neighbour of {0}'.format(
            person
        )
    )
    for person in people
]

In [11]:
solver.AddConstraint(
    solver.AllDifferent(neighbour_vars)
)

In [12]:
for constraint in constraints:
    first_person, second_person = constraint
    solver.AddConstraint(
        neighbour_vars[person_index[first_person]] !=
        person_index[second_person]
    )
    solver.AddConstraint(
        neighbour_vars[person_index[second_person]] !=
        person_index[first_person]
    )

In [13]:
for index, neighbor_var in enumerate(neighbour_vars):
    solver.AddConstraint(
        solver.Element(neighbour_vars, neighbor_var) == index
    )

In [14]:
number_of_rooms = solver.IntVar(0, n, 'number of rooms')  # Variable to optimize

In [15]:
solver.AddConstraint(
    number_of_rooms == solver.Sum([
        neighbor_var == index
        for index, neighbor_var in enumerate(neighbour_vars)
    ])
)

In [18]:
solution = solver.Assignment()
solution.Add(neighbour_vars)
solution.Add(number_of_rooms)

<ortools.constraint_solver.pywrapcp.IntVarElement; proxy of <Swig Object of type 'operations_research::IntVarElement *' at 0x10c0a3ed0> >

In [19]:
objective = solver.Minimize(
    number_of_rooms, 4
)

In [20]:
solution_collector = solver.AllSolutionCollector(solution)  # We need it to collect all of the solutions

In [21]:
solver.Solve(solver.DefaultPhase(neighbour_vars), [solution_collector, objective])

True

In [24]:
num_solutions = solution_collector.SolutionCount()
print(num_solutions)

6


In [25]:
for solution_num in range(num_solutions):
    print('Solution #{0}'.format(solution_num + 1))
    room_number = 1
    for index, neighbor_var in enumerate(neighbour_vars):
        first_person = index
        neighbour = solution_collector.Value(solution_num, neighbor_var)
        if first_person < neighbour:
            print(
                'Room #{0}: {1}, {2}'.format(
                    room_number, index_to_person[first_person],
                    index_to_person[neighbour]
                )
            )
            room_number += 1
        if first_person == neighbour:
            print(
                'Room #{0}: {1}'.format(
                    room_number, index_to_person[first_person]
                )
            )
            room_number += 1

Solution #1
Room #1: Alexey Alekseev
Room #2: Alexey Mironov
Room #3: Alexey Eskov
Room #4: Alexey Evseev
Room #5: Anton Rodin
Room #6: Arshdeep Singh
Room #7: Egor Sachko
Room #8: Erik Hinterbichler
Room #9: Danyi Gergely
Room #10: Ihor Bykov
Room #11: Ilya Tikhonov
Room #12: Kirill Klokov
Room #13: Luis del Giudice Sierra
Room #14: Mikael Arakelian
Room #15: Nikolay Fokin
Room #16: Oleg Khomyuk
Room #17: Oleksii Solianik
Room #18: Owais Lone
Room #19: Spyridon Oikonomakis
Room #20: Tigran Najaryan
Room #21: Vadim Goncharov
Solution #2
Room #1: Alexey Alekseev
Room #2: Alexey Mironov
Room #3: Alexey Eskov, Erik Hinterbichler
Room #4: Alexey Evseev
Room #5: Anton Rodin
Room #6: Arshdeep Singh
Room #7: Egor Sachko
Room #8: Danyi Gergely
Room #9: Ihor Bykov
Room #10: Ilya Tikhonov
Room #11: Kirill Klokov
Room #12: Luis del Giudice Sierra
Room #13: Mikael Arakelian
Room #14: Nikolay Fokin
Room #15: Oleg Khomyuk, Owais Lone
Room #16: Oleksii Solianik
Room #17: Spyridon Oikonomakis
Room #18